In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("/content/drive/MyDrive/CMPT733_Final_Project")
!ls

 covid_19_data.csv
 covid-19_global_daily.csv
 covid-19_us_daily.csv
 Covid-19_visualization.ipynb
 covid_tweet
 covid_tweet.csv
 data_analysis_visulization
'Final Presentation.gslides'
 Find_Other_Tags.ipynb
 graphs
 ieeeset_extraction.ipynb
 JKU_repo_ETL.ipynb
 Kevin_Milestone_Presentation.m4a
 media_accounts.txt
 Media_Analysis.ipynb
 milestone_presentation.docx
 Milestone_with_SFU_ppt_template.pptx
'New Recording 3.m4a'
 other_tags.csv
 preprocess_visualization_covid19_us_2.ipynb
 preprocess_visualization_covid19_us.ipynb
'Project Report.gdoc'
 Project_Timeline.xlsx
 project_twitter_gettingdata.ipynb
 Sentiment_Analysis_v1.ipynb
 Sentiment_Analysis_Visualization.ipynb
 SFU-Powerpoint-Template-Standard.pptx
 state-geocodes-v2016.xls
 Tag_Clustering.ipynb
 tag_score_prediction.ipynb
 tag_score_prediction_v2.ipynb
 tweet_sentiment_added
 tweet_sentiment_added_noise_removed
 us_counties_covid19_daily.csv
 us_covid19_daily.csv
 us_states_covid19_daily.csv


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Covid-19 Data of Each Contry and Region, ETL from Global Time Series

In [ ]:
# time series
confirmed_us = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
deaths_us = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

confirmed_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

### Fix Turcky data ( by some data from other dataset)  

In [ ]:
data = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,3,14,17,50,129,268,521,978,1824,2577,3363,4160,5093,6619,9872,15500,20135,25072,29451,36806,42649,49329,56907,65103,73630,82193,92779,103977,115010,127922,141898,154924,166057,177106,188751,201810,213650,223940,234758,247472,260014,268400,276876,285368,293150,299561,305357,311863,319849,326962,332913,338307,342849,347239,352222,358350,363728,368755,372960,377154,380184,384819,389277,393724,398370,402749,406470,409620,412400,415044,417658,420247,423473,426577,429262,431841,434656,437871,440975,443649,445931,448180,450318,452676,455363,457893,460281,462767,465457,468158,470666,473351,476601,480570,484819,489149,493139,497026,500573,503875,507270,510512,513809,517258,521316,525282,529079,532811,536499,540236,543753,546995,550221,553409,556548,559671,562625,565489,568321,571106,573834,576598,579351,582093,584791,587367,589905,592424,594921,597434,599966,602490,604943,607426,609975,612480,615001,617501,620120,622682,625312,627983,630692,633377,636083,639029,642233,645369,648592,651780,654995,658240,661458,664755,668136,671471,674887,678129,681483,684918,688462,692303,695575,699136,702446,706371,710457,714028,718084,722210,726423,730454,734771,739047,743388,747854,752239,756790,761082,765714,770504,775055,779168,783713,787818,791972,796640,801378,806195,810678,815495,819678,823810,828551,833153,837959,842640,847169,851279,855269,859110,862992,866776,870603,874525,878611,882498,886858,890968,895268,899661,904092,908577,912663,917053,921492,926037,930642,935571,940258,945195,950706,955858,961333,967051,972940,978628,984114,990093,996102,1002372,1008680,1014996,1021015,1026743,1033005,1039378,1045882,1052168,1058794,1065548,1072392,1079399,1086278,1093603,1101331,1109614,1118090,1126857,1135877,1146265,1157730,1170084,1183964,1199011,1215378,1233638,1253717,1282068,1311200,1341045,1371148,1400429,1431648,1461758,1493681,1526062,1558798,1590694,1621096,1653233,1686431,1718143,1748567]
len(data)

300

In [ ]:
confirmed_global.iloc[248, 28:328] = data
confirmed_global.iloc[248, 317:328]

In [ ]:
confirmed_global[confirmed_global['Country/Region'] == 'Turkey'].loc[:, "2/15/20":"12/10/20"]

,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,...,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20
248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,14.0,17.0,50.0,129.0,268.0,521.0,978.0,1824.0,2577.0,3363.0,4160.0,5093.0,6619.0,...,1026743.0,1033005.0,1039378.0,1045882.0,1052168.0,1058794.0,1065548.0,1072392.0,1079399.0,1086278.0,1093603.0,1101331.0,1109614.0,1118090.0,1126857.0,1135877.0,1146265.0,1157730.0,1170084.0,1183964.0,1199011.0,1215378.0,1233638.0,1253717.0,1282068.0,1311200.0,1341045.0,1371148.0,1400429.0,1431648.0,1461758.0,1493681.0,1526062.0,1558798.0,1590694.0,1621096.0,1653233.0,1686431.0,1718143.0,1748567


### Combine Comfirmed, Deaths, Recovered, Active data

In [ ]:
dfs = [confirmed_global, deaths_global, recovered_global]
colnames = ['Comfirmed', 'Deaths', 'Recovered']
daily_global = pd.DataFrame(columns=["Date", 'Country/Region'])
for i in range(3):
    group_data = dfs[i].groupby('Country/Region').sum()
    confirmed_global_1 = group_data[group_data.columns[2:]]
    temp_melt = confirmed_global_1.reset_index().melt(id_vars="Country/Region", var_name="Date", value_name=colnames[i])
    daily_global = daily_global.merge(temp_melt, how='outer', on=['Date', 'Country/Region'])
daily_global

,Country/Region,Date,Comfirmed,Deaths,Recovered
0,Afghanistan,1/22/20,0.0,0,0
1,Albania,1/22/20,0.0,0,0
2,Algeria,1/22/20,0.0,0,0
3,Andorra,1/22/20,0.0,0,0
4,Angola,1/22/20,0.0,0,0
...,...,...,...,...,...
85243,Vietnam,4/9/21,2683.0,35,2429
85244,West Bank and Gaza,4/9/21,264395.0,2812,229876
85245,Yemen,4/9/21,5233.0,1022,1987
85246,Zambia,4/9/21,89783.0,1226,85559


In [ ]:
daily_global['Active'] = daily_global['Comfirmed'] - daily_global['Deaths'] - daily_global['Recovered']
daily_global

,Country/Region,Date,Comfirmed,Deaths,Recovered,Active
0,Afghanistan,1/22/20,0.0,0,0,0.0
1,Albania,1/22/20,0.0,0,0,0.0
2,Algeria,1/22/20,0.0,0,0,0.0
3,Andorra,1/22/20,0.0,0,0,0.0
4,Angola,1/22/20,0.0,0,0,0.0
...,...,...,...,...,...,...
85243,Vietnam,4/9/21,2683.0,35,2429,219.0
85244,West Bank and Gaza,4/9/21,264395.0,2812,229876,31707.0
85245,Yemen,4/9/21,5233.0,1022,1987,2224.0
85246,Zambia,4/9/21,89783.0,1226,85559,2998.0


In [ ]:
group_data = confirmed_global.groupby('Country/Region').sum()
daily_increase_global = group_data[group_data.columns[2:]].diff(axis=1).fillna(0).astype(int)
temp_melt = daily_increase_global.reset_index().melt(id_vars="Country/Region", var_name="Date", value_name='Daily_Increase')
daily_global = daily_global.merge(temp_melt, how='outer', on=['Date', 'Country/Region'])
daily_global

,Country/Region,Date,Comfirmed,Deaths,Recovered,Active,Daily_Increase
0,Afghanistan,1/22/20,0.0,0,0,0.0,0
1,Albania,1/22/20,0.0,0,0,0.0,0
2,Algeria,1/22/20,0.0,0,0,0.0,0
3,Andorra,1/22/20,0.0,0,0,0.0,0
4,Angola,1/22/20,0.0,0,0,0.0,0
...,...,...,...,...,...,...,...
85243,Vietnam,4/9/21,2683.0,35,2429,219.0,15
85244,West Bank and Gaza,4/9/21,264395.0,2812,229876,31707.0,2378
85245,Yemen,4/9/21,5233.0,1022,1987,2224.0,100
85246,Zambia,4/9/21,89783.0,1226,85559,2998.0,191


In [ ]:
daily_global.to_csv('covid-19_global_daily.csv')

# Covid-19 Data of Each States in US, ETL from US Daily Reports

In [ ]:
population_us = deaths_us.groupby('Province_State').sum()['Population'].reset_index()
population_us

,Province_State,Population
0,Alabama,4903185
1,Alaska,740995
2,American Samoa,55641
3,Arizona,7278717
4,Arkansas,3017804
5,California,39512223
6,Colorado,5758736
7,Connecticut,3565287
8,Delaware,973764
9,Diamond Princess,0


In [ ]:
# catch data structure's change
datelist = pd.date_range(start="2020-04-12",end="2021-02-12").tolist()
last_columns = None
for date in datelist:
    # date = date.strftime("%m-%d-%Y")
    csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/' + date.strftime("%m-%d-%Y") + '.csv'
    df = pd.read_csv(csv_url)
    if df.columns.to_list() != last_columns:
        print(date.strftime("%m-%d-%Y"), ':', df.columns.to_list())
    last_columns = df.columns.to_list()

04-12-2020 : ['Province_State', 'Country_Region', 'Last_Update', 'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'FIPS', 'Incident_Rate', 'People_Tested', 'People_Hospitalized', 'Mortality_Rate', 'UID', 'ISO3', 'Testing_Rate', 'Hospitalization_Rate']
11-09-2020 : ['Province_State', 'Country_Region', 'Last_Update', 'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'FIPS', 'Incident_Rate', 'Total_Test_Results', 'People_Hospitalized', 'Case_Fatality_Ratio', 'UID', 'ISO3', 'Testing_Rate', 'Hospitalization_Rate']


In [ ]:
# date
last_df = None
daily_us = pd.DataFrame()
frames = []
datelist = pd.date_range(start="2020-04-12",end="2021-02-12").tolist()
# datelist = pd.date_range(start="2020-04-12",end="2020-04-15").tolist()
for date in datelist:
    # date = date.strftime("%m-%d-%Y")
    csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/' + date.strftime("%m-%d-%Y") + '.csv'
    df = pd.read_csv(csv_url)
    df.columns = ['Province_State', 'Country_Region', 'Last_Update', 'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'FIPS', 'Incident_Rate', 'Total_Test_Results', 'People_Hospitalized', 'Case_Fatality_Ratio', 'UID', 'ISO3', 'Testing_Rate', 'Hospitalization_Rate']
    df = df.merge(population_us, on='Province_State', how='right') # add population of each state 
    df['Date'] = date
    df.sort_values(by=["Province_State"])
    if last_df is not None:
        df['Daily_Increase'] = df['Confirmed'] - last_df['Confirmed']
        df['Daily_Tested'] = df['Total_Test_Results'] - last_df['Total_Test_Results']
    else:
        df['Daily_Increase'] = df['Confirmed']
        df['Daily_Tested'] = df['Total_Test_Results'] 

    frames.append(df)
    last_df = df


daily_us = pd.concat(frames)
daily_us.set_index(['Date', 'Province_State'], inplace=True)


In [ ]:
daily_us

Country_Region  ... Daily_Tested
Date       Province_State                 ...             
2020-04-12 Alabama                    US  ...      21583.0
           Alaska                     US  ...       8038.0
           American Samoa             US  ...          3.0
           Arizona                    US  ...      52289.0
           Arkansas                   US  ...      19722.0
...                                  ...  ...          ...
2021-02-12 Virginia                   US  ...      35220.0
           Washington                 US  ...      25411.0
           West Virginia              US  ...      14322.0
           Wisconsin                  US  ...      29891.0
           Wyoming                    US  ...       3475.0

[17806 rows x 20 columns]

In [ ]:
daily_us["Daily_Pos_over_Tested"] = daily_us['Daily_Increase'] * 100 / daily_us['Daily_Tested']
daily_us["Confirmed_Percentage"] = daily_us['Confirmed'] * 100 / daily_us['Population']
daily_us.to_csv('covid-19.csv')